<div style="font-size: 22px; line-height: 1.6;">

# 🌊 Marine Carbonate Visualizer - Bjerrum Plot

<div style="text-align: center; margin: 30px 0;">
    <img src="figures/carbonate_system.svg" width="800" alt="Marine Carbonate System Diagram" style="border-radius: 10px; box-shadow: 0 4px 8px rgba(0,0,0,0.1); max-width: 100%;">
</div>

<div style="text-align: center; font-size: 18px; color: #00204C; margin: 20px 0;">
    <strong>? Interactive Bjerrum diagram for carbonate system analysis</strong>
</div>

</div>

<div style="font-size: 22px; line-height: 1.6;">

---

## 📘 *Interactive Bjerrum Plot for Carbonate Systems*

This tool provides **Bjerrum plot visualization** showing how changes in **Total Alkalinity (TA)** and **Dissolved Inorganic Carbon (DIC)** affect the marine carbonate system speciation, using the Python package **PyCO2SYS**.

---

#### 📊 What does the Bjerrum plot show?

**Input parameters:**
* **Total alkalinity** (TA) in µmol/kg
* **Dissolved Inorganic Carbon** (DIC) in µmol/kg

**Fixed conditions:**
* **Salinity** = 35 PSU
* **Temperature** = 25 °C

**Interactive visualizations:**
1. **📈 Bjerrum Plot** - Species fractions (α) vs pH showing CO₂, HCO₃⁻, and CO₃²⁻ distribution
2. **🥧 Current Composition** - Pie chart of species at current pH
3. **📊 pH-pCO₂ Relationship** - How pH affects partial pressure of CO₂
4. **📋 Complete Analysis** - Detailed numerical results and system parameters

**Educational Value:**
- Visualize chemical equilibria in seawater
- Understand buffer capacity and pH control
- Explore ocean acidification effects
- Analyze calcification potential

</div>

In [1]:
import PyCO2SYS as pyco2

def compute_carbonate_system(alkalinity, DIC, salinity=35, temperature=25):
    # Run CO2SYS using alkalinity and DIC as input parameters
    results = pyco2.sys(
        par1=alkalinity,      # par1_type = 1 → Total Alkalinity (µmol/kg)
        par2=DIC,             # par2_type = 2 → DIC = Dissolved Inorganic Carbon (µmol/kg)
        par1_type=1,          # 1 = total alkalinity
        par2_type=2,          # 2 = DIC
        salinity=salinity,    # Salinity in PSU
        temperature=temperature  # Temperature in °C
    )

    # Return selected key results in a dictionary
    return {
        "pH_total": results["pH_total"],         # pH on the total scale
        "pCO2": results["pCO2"],                 # partial pressure of CO₂ (µatm)
        "bicarbonate": results["bicarbonate"],   # HCO₃⁻ concentration (µmol/kg)
        "carbonate": results["carbonate"],       # CO₃²⁻ concentration (µmol/kg)
        "omega_aragonite": results["saturation_aragonite"],  # saturation state with respect to aragonite
        "salinity": salinity,
        "temperature": temperature
    }



In [2]:
from IPython.display import display, HTML

def show_conditions(data):
    html = f"""
    <div style="font-size: 20px; color: #00204C; font-family: sans-serif; margin-top: 15px;">
        <p><strong>🧪 Conditions used in the calculation:</strong></p>
        <p>🔹 Salinity: {data['salinity']:.0f} PSU</p>
        <p>🌡️ Temperature: {data['temperature']:.0f} °C</p>
    </div>
    """
    display(HTML(html))


In [3]:
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML

def create_bjerrum_plot(data):
    """Create a Bjerrum plot showing carbonate system speciation vs pH"""
    
    # Set up the plotting style
    plt.style.use('default')
    
    # Create figure with subplots
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Marine Carbonate System - Bjerrum Plot Analysis', fontsize=22, fontweight='bold', y=0.98)
    
    # Colors using viridis colormap
    viridis = plt.cm.viridis
    colors = [viridis(0.15), viridis(0.45), viridis(0.75), viridis(0.9), viridis(0.6)]
    
    # Set figure background
    fig.patch.set_facecolor('white')
    
    # Generate pH range for Bjerrum plot
    pH_range = np.linspace(4, 11, 100)
    
    # Calculate speciation for each pH value using simplified constants
    co2_fractions = []
    hco3_fractions = []
    co3_fractions = []
    
    # Carbonic acid dissociation constants (at 25°C, S=35)
    K1 = 10**(-6.0)   # First dissociation constant 
    K2 = 10**(-9.3)   # Second dissociation constant 
    
    for pH in pH_range:
        H = 10**(-pH)
        
        # Alpha values for CO2, HCO3-, CO3-2
        denominator = H**2 + K1*H + K1*K2
        alpha0 = H**2 / denominator  # CO2
        alpha1 = K1*H / denominator  # HCO3-
        alpha2 = K1*K2 / denominator  # CO3-2
        
        co2_fractions.append(alpha0)
        hco3_fractions.append(alpha1)
        co3_fractions.append(alpha2)
    
    # Plot 1: Bjerrum Plot - Species fractions vs pH
    ax1.plot(pH_range, co2_fractions, color=colors[0], linewidth=4, label='CO₂ (H₂CO₃)', alpha=0.9)
    ax1.plot(pH_range, hco3_fractions, color=colors[1], linewidth=4, label='HCO₃⁻', alpha=0.9)
    ax1.plot(pH_range, co3_fractions, color=colors[2], linewidth=4, label='CO₃²⁻', alpha=0.9)
    
    # Mark current pH
    current_pH = data['pH_total']
    ax1.axvline(x=current_pH, color='red', linestyle='--', linewidth=3, alpha=0.8)
    ax1.text(current_pH + 0.15, 0.85, f'Current\npH: {current_pH:.2f}', 
             fontsize=13, fontweight='bold', color='red',
             bbox=dict(boxstyle="round,pad=0.4", facecolor='white', alpha=0.9, edgecolor='red', linewidth=1.5))
    
    ax1.set_xlabel('pH', fontsize=16, fontweight='bold')
    ax1.set_ylabel('Fraction of Total DIC (α)', fontsize=16, fontweight='bold')
    ax1.set_title('Bjerrum Plot: Carbonate Speciation vs pH', fontsize=18, fontweight='bold', pad=20)
    ax1.grid(True, alpha=0.4, linestyle='-', linewidth=0.5)
    ax1.legend(fontsize=13, loc='center right', frameon=True, fancybox=True, shadow=True)
    ax1.set_xlim(4, 11)
    ax1.set_ylim(0, 1)
    ax1.tick_params(axis='both', which='major', labelsize=12)
    
    # Plot 2: Current system composition (pie chart)
    current_co2 = data["pCO2"] * 0.034
    species_names = ['CO₂(aq)', 'HCO₃⁻', 'CO₃²⁻']
    species_values = [current_co2, data["bicarbonate"], data["carbonate"]]
    
    wedges, texts, autotexts = ax2.pie(species_values, labels=species_names, autopct='%1.1f%%', 
            colors=colors[0:3], startangle=90, textprops={'fontsize': 12, 'fontweight': 'bold'},
            wedgeprops=dict(width=0.8, edgecolor='white', linewidth=2))
    
    for autotext in autotexts:
        autotext.set_color('white')
        autotext.set_fontsize(11)
        autotext.set_fontweight('bold')
    
    ax2.set_title(f'Current Composition at pH {current_pH:.2f}', fontweight='bold', fontsize=16, pad=20)
    
    # Plot 3: Saturation state visualization
    omega_val = data["omega_aragonite"]
    categories = ['Undersaturated\n(Ω < 1)', 'Saturated\n(Ω ≥ 1)']
    
    if omega_val < 1:
        values = [omega_val, 0]
        bar_colors = [viridis(0.2), 'lightgray']
    else:
        values = [0, omega_val]
        bar_colors = ['lightgray', viridis(0.8)]
    
    bars = ax3.bar(categories, values, color=bar_colors, alpha=0.8, edgecolor='black', linewidth=1.5)
    ax3.set_ylabel('Ωₐ value', fontsize=14, fontweight='bold')
    ax3.set_title('Aragonite Saturation State', fontsize=16, fontweight='bold', pad=20)
    ax3.axhline(y=1, color='black', linestyle='--', alpha=0.7, linewidth=2)
    ax3.text(0.5, max(values)*0.7, f'Ωₐ = {omega_val:.2f}', 
             ha='center', fontsize=15, fontweight='bold',
             bbox=dict(boxstyle="round,pad=0.4", facecolor=colors[4], alpha=0.8, edgecolor='black', linewidth=1))
    ax3.grid(True, alpha=0.4, axis='y', linestyle='-', linewidth=0.5)
    ax3.tick_params(axis='both', which='major', labelsize=11)
    
    # Plot 4: System parameters summary
    ax4.axis('off')
    
    # Calculate total DIC
    total_dic = data['bicarbonate'] + data['carbonate'] + current_co2
    
    conditions_text = f"""
System Parameters:

Input Conditions:
• Total Alkalinity: {(data['bicarbonate'] + 2*data['carbonate']):.0f} µmol/kg
• DIC Total: {total_dic:.0f} µmol/kg
• Salinity: {data['salinity']:.0f} PSU
• Temperature: {data['temperature']:.0f} °C

Calculated Results:
• pH (total): {data['pH_total']:.2f}
• pCO₂: {data['pCO2']:.0f} µatm
• [CO₂]: {current_co2:.1f} µmol/kg
• [HCO₃⁻]: {data['bicarbonate']:.0f} µmol/kg
• [CO₃²⁻]: {data['carbonate']:.0f} µmol/kg

Saturation:
• Omega_a (aragonite): {data['omega_aragonite']:.2f}
"""
    
    ax4.text(0.05, 0.95, conditions_text, fontsize=12, verticalalignment='top', fontweight='normal',
             bbox=dict(boxstyle="round,pad=0.6", facecolor=viridis(0.05), alpha=0.1, 
                      edgecolor=viridis(0.3), linewidth=1.5),
             transform=ax4.transAxes)
    
    plt.subplots_adjust(hspace=0.4, wspace=0.25, top=0.93)
    plt.tight_layout()
    plt.show()
    
    return fig

In [4]:
from ipywidgets import FloatSlider, Layout, HTML, VBox, interact, Button
from IPython.display import display

# Slider style settings - increased spacing for labels
slider_style = {'description_width': '350px'}
slider_layout = Layout(width='950px', margin='0 0 20px 0', justify_content='flex-start')

# Alkalinity slider
alk_slider = FloatSlider(
    value=2300, min=1800, max=3500, step=25,
    description="Alkalinity (µmol/kg)",
    style=slider_style, layout=slider_layout,
    continuous_update=False,
    readout=True, readout_format=".0f"
)

# DIC slider
dic_slider = FloatSlider(
    value=2020, min=1600, max=3500, step=25,
    description="DIC (µmol/kg)",
    style=slider_style, layout=slider_layout,
    continuous_update=False,
    readout=True, readout_format=".0f"
)

# Custom CSS for widgets
custom_css = HTML("""
<style>
.widget-label {
    font-size: 18pt !important;
    font-weight: bold;
    color: #00204C !important;
}
.widget-readout {
    font-size: 20pt !important;
    font-weight: bold;
    color: #00204C !important;
}
input[type="range"] {
    height: 20px !important;
    background: linear-gradient(to right, #00204C, #FDE725) !important;
    border-radius: 10px;
}
</style>
""")

# Create a reset button
reset_button = Button(
    description='Reset Values',
    button_style='info',
    layout=Layout(width='150px', margin='10px 0 0 0')
)

def reset_values(b):
    alk_slider.value = 2300
    dic_slider.value = 2020

reset_button.on_click(reset_values)


In [5]:
from ipywidgets import Output, VBox, interactive_output, Button, Layout
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

# Create output widget to display results
output = Output()

# Function to update display when sliders change
def update_display(alkalinity, DIC):
    # Compute carbonate system variables using current slider values
    data = compute_carbonate_system(alkalinity, DIC)
    
    # Clear and update output
    with output:
        clear_output(wait=True)
        
        # Create and display the Bjerrum plot
        create_bjerrum_plot(data)

# Function to reset sliders to default values
def reset_values(button):
    alk_slider.value = 2300
    dic_slider.value = 2020
    with output:
        clear_output(wait=True)
    update_display(alk_slider.value, dic_slider.value)

# Create reset button
reset_button = Button(
    description="Reset Values",
    button_style='info',
    layout=Layout(width='200px', margin='10px 0')
)
reset_button.on_click(reset_values)

# Create interactive output that connects sliders to update function
interactive_plot = interactive_output(update_display, {'alkalinity': alk_slider, 'DIC': dic_slider})

# Display custom CSS styles for consistent font and slider appearance
display(custom_css)

# Create the interface layout
interface = VBox([
    alk_slider,
    dic_slider,
    reset_button,
    output
])

# Display the interface
display(interface)

# Trigger initial update
update_display(alk_slider.value, dic_slider.value)

HTML(value='\n<style>\n.widget-label {\n    font-size: 18pt !important;\n    font-weight: bold;\n    color: #0…

**Author:** Cardoso-Mohedano JG 
**Institution:** Instituto de Ciencias del Mar y Limnología, UNAM, Estación El Carmen  
**License:** [CC BY-NC 4.0](https://creativecommons.org/licenses/by-nc/4.0/)  
**ORCID:** [0000-0002-2918-972X](https://orcid.org/0000-0002-2918-972X)

---
This interactive tool allows exploration of the marine carbonate system using [PyCO2SYS](https://pyco2sys.readthedocs.io/en/latest/).  
Developed with support from [Claude AI](https://claude.ai) by Anthropic and [OpenAI ChatGPT](https://openai.com/chatgpt) and educational Python tools.
